# Stanford CoreNLP-processed summaries

The idea here is to first explore this processed data, and extract meaningful metrics to quantify the importance of women characters in movies, and movies regarding a feminism score.

One first idea would to associate a score based on how many entries appear for each character in the processed data, then weight it based on the actual processed data (state of the character regarding the word associated with it, importance of the words, etc.)